In [ ]:
import oracledb
import os
import pandas as pd
from dotenv import load_dotenv
import numpy as np
import logging
import os
import glob

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def get_oracle_type(dtype):
    """Map pandas dtypes to Oracle data types"""
    if pd.api.types.is_integer_dtype(dtype):
        return "NUMBER"
    elif pd.api.types.is_float_dtype(dtype):
        return "NUMBER"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "TIMESTAMP"
    elif pd.api.types.is_string_dtype(dtype):
        return "VARCHAR2(500)"
    else:
        return "VARCHAR2(500)"  # Default to VARCHAR2 for unknown types

def clean_value(val):
    """Convert pandas/numpy types to Python native types for Oracle insertion"""
    if pd.isna(val):
        return None
    elif isinstance(val, (np.int64, np.int32)):
        return int(val)
    elif isinstance(val, (np.float64, np.float32)):
        return float(val)
    elif isinstance(val, pd.Timestamp):
        return val.strftime('%Y-%m-%d %H:%M:%S')
    return val

def connect_to_oracle():
    """Establishes a connection to the Oracle database"""
    load_dotenv()
    required_vars = ["DB_USER", "DB_PASSWORD", "DB_DSN", "ORACLE_CLIENT_LIB_DIR", "ORACLE_CONFIG_DIR"]
    
    for var in required_vars:
        if os.getenv(var) is None:
            raise EnvironmentError(f"Environment variable '{var}' not set")

    oracledb.init_oracle_client(
        lib_dir=os.getenv("ORACLE_CLIENT_LIB_DIR"),
        config_dir=os.getenv("ORACLE_CONFIG_DIR")
    )

    return oracledb.connect(
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        dsn=os.getenv("DB_DSN")
    )

def delete_table(connection, table_to_delete):
    """Deletes the specified table from Oracle database"""
    with connection.cursor() as cursor:
        try:
            cursor.execute(f"DROP TABLE {table_to_delete} PURGE")
            logging.info(f"Table {table_to_delete} deleted successfully.")
        except oracledb.DatabaseError as e:
            logging.error(f"Error deleting table {table_to_delete}: {e}")

def prepare_data(data):
    """Prepares data by converting date columns and cleaning column names"""
    # Convert date columns
    if 'H_DATETIME' in data.columns:
        data['H_DATETIME'] = pd.to_datetime(data['H_DATETIME'], format='%d.%m.%Y %H:%M', errors='coerce')
    if 'A_DATETIME' in data.columns:
        data['A_DATETIME'] = pd.to_datetime(data['A_DATETIME'], format='%d.%m.%Y %H:%M', errors='coerce')
    
    # Clean and format column names for Oracle
    data.columns = data.columns.str.replace('[^0-9a-zA-Z]+', '_', regex=True).str.upper()
    return data

def create_table(cursor, data, table_name):
    """Creates a new table in Oracle with appropriate columns based on the DataFrame schema"""
    columns_def = [f"{col} {get_oracle_type(data[col].dtype)}" for col in data.columns]
    create_table_sql = f"CREATE TABLE {table_name} ({', '.join(columns_def)})"
    logging.info(f"Creating table {table_name} with SQL:\n{create_table_sql}")
    cursor.execute(create_table_sql)

def insert_data_batch(cursor, data, table_name, batch_size=1000):
    """Inserts data into the table in batches"""
    columns = ', '.join(data.columns)
    placeholders = ', '.join([f":{i + 1}" for i in range(len(data.columns))])
    insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    rows_to_insert = [tuple(clean_value(row[col]) for col in data.columns) for _, row in data.iterrows()]
    
    # Insert in batches
    successful_inserts = 0
    for start in range(0, len(rows_to_insert), batch_size):
        batch = rows_to_insert[start:start + batch_size]
        try:
            cursor.executemany(insert_sql, batch)
            successful_inserts += len(batch)
            logging.info(f"Inserted {successful_inserts} rows so far...")
        except Exception as e:
            logging.error(f"Failed to insert batch at row {start}: {e}")

def create_and_upload(connection, csv_file_path, table_name):
    """Creates a table and uploads data from a CSV file to the Oracle database"""
    data = pd.read_csv(csv_file_path).loc[:, ~pd.read_csv(csv_file_path).columns.str.contains('^Unnamed')]
    data = prepare_data(data)
    
    with connection.cursor() as cursor:
        delete_table(connection, table_name)
        create_table(cursor, data, table_name)
        
        # Insert data in batches
        insert_data_batch(cursor, data, table_name)
        
        # Final commit after batch insertions
        connection.commit()
        logging.info(f"Upload to table {table_name} completed.")

def show_table(connection, table_name):
    """Fetches and displays data from the specified Oracle table"""
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
    return pd.DataFrame(rows, columns=column_names)

def list_htdatan_tables(connection) -> pd.DataFrame:
    """Retrieves a list of user tables in the Oracle database that match specific naming conventions."""
    query = """
    SELECT TABLE_NAME
    FROM ALL_TABLES
    WHERE REGEXP_LIKE(TABLE_NAME, '^(B_|L1_|LL_|PL_|SA_)')
    ORDER BY TABLE_NAME
    """

    with connection.cursor() as cursor:
        cursor.execute(query)
        tables = cursor.fetchall()
        table_names = [table[0] for table in tables]

    logging.info(f"Retrieved {len(table_names)} matching tables from the database.")
    return pd.DataFrame(table_names, columns=["TABLE_NAME"])

# connection = connect_to_oracle()
# tables_df = list_htdatan_tables(connection)
# tables_df

In [31]:
# #  update csvs as tables into oracle db

# Define the folder path containing the CSV files
csv_folder_path = r"C:\Users\gebel\github\football_stats\data\htdatan"
csv_files = glob.glob(os.path.join(csv_folder_path, "*.csv"))
# csv_files = ['C:\\Users\\gebel\\github\\football_stats\\data\\htdatan\\sa_2425.csv']

In [40]:
# #  update csvs as tables into oracle db

# Define the folder path containing the CSV files
csv_folder_path = r"C:\Users\gebel\github\football_stats\data\xg"
csv_files = glob.glob(os.path.join(csv_folder_path, "*.csv"))
# csv_files = ['C:\\Users\\gebel\\github\\football_stats\\data\\htdatan\\sa_2425.csv']

In [41]:
for csv_file_path in csv_files:
    table_name = os.path.splitext(os.path.basename(csv_file_path))[0].upper()
    print(table_name)

BUNDESLIGA2021
BUNDESLIGA2022
BUNDESLIGA2023
BUNDESLIGA2024
EPL2021
EPL2022
EPL2023
EPL2024
LA_LIGA2021
LA_LIGA2022
LA_LIGA2023
LA_LIGA2024
LIGUE_12021
LIGUE_12022
LIGUE_12023
LIGUE_12024
SERIE_A2021
SERIE_A2022
SERIE_A2023
SERIE_A2024


In [53]:
csv_files[3]

'C:\\Users\\gebel\\github\\football_stats\\data\\xg\\bundesliga2024.csv'

In [ ]:
# read csv as pd df
df = pd.read_csv(csv_files[11]).loc[:, ~pd.read_csv(csv_files[0]).columns.str.contains('^Unnamed')]
print(len( df ) )
# df_grouped = df.groupby(['gameurl']).size().reset_index(name='count')
# df_grouped
df[df["TEAMS"]=="Alaves"]

113


,TEAMS,GOALS,xG,SHOTS,SHOTS ON TARGET,DEEP,PPDA,xPTS,A_TEAMS,A_GOALS,A_xG,A_SHOTS,A_SHOTS ON TARGET,A_DEEP,A_PPDA,A_xPTS,timing_chart_xg,gameurl,id
47,Alaves,2,2.01,15,5,6,16.00,2.51,Las Palmas,0,0.55,12,6,7,6.56,0.33,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27017,48
54,Alaves,0,0.79,11,2,3,27.82,0.32,Barcelona,3,2.49,14,9,9,4.92,2.55,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27064,55
55,Alaves,0,0.17,5,1,2,10.47,0.36,Real Betis,0,1.36,15,3,5,7.28,2.40,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27000,56
61,Alaves,2,1.41,10,4,4,13.58,2.17,Sevilla,1,0.62,8,2,3,7.62,0.52,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27035,62
92,Alaves,2,0.43,10,4,5,9.36,0.17,Real Valladolid,3,2.34,12,4,1,16.15,2.75,0' Total xG: 0.00;0' Total xG: 0.02;0' Total x...,https://understat.com/match/27078,93
108,Alaves,1,1.44,13,3,8,5.25,2.03,Mallorca,0,0.71,7,2,3,8.00,0.70,0' Total xG: 0.00;0' Total xG: 0.00;0' Total x...,https://understat.com/match/27097,109


In [ ]:
# create and upload the csvs as tables in oracle
for csv_file_path in csv_files:
    table_name = os.path.splitext(os.path.basename(csv_file_path))[0].upper()
    
    # Establish a connection and upload the CSV file as a table in Oracle
    try:
        with connect_to_oracle() as connection:
            create_and_upload(connection, csv_file_path, table_name)
            logging.info(f"Successfully uploaded {csv_file_path} to table {table_name}")
    except Exception as e:
        logging.error(f"Failed to upload {csv_file_path} to table {table_name}: {e}")


2024-11-03 01:03:45,097 - INFO - Table SA_2425 deleted successfully.
2024-11-03 01:03:45,100 - INFO - Creating table SA_2425 with SQL:
CREATE TABLE SA_2425 (H_DATETIME VARCHAR2(500), H_GAMEINFO VARCHAR2(500), H_TEAMNAMES VARCHAR2(500), H_GOALS NUMBER, H_BALL_POSSESSION NUMBER, H_GOAL_ATTEMPTS NUMBER, H_SHOTS_ON_GOAL NUMBER, H_SHOTS_OFF_GOAL NUMBER, H_BLOCKED_SHOTS NUMBER, H_FREE_KICKS NUMBER, H_CORNER_KICKS NUMBER, H_OFFSIDES NUMBER, H_GOALKEEPER_SAVES NUMBER, H_FOULS NUMBER, H_YELLOW_CARDS NUMBER, H_RED_CARDS NUMBER, H_TOTAL_PASSES NUMBER, H_COMPLETED_PASSES NUMBER, H_TACKLES NUMBER, H_ATTACKS NUMBER, H_DANGEROUS_ATTACKS NUMBER, A_DATETIME VARCHAR2(500), A_GAMEINFO VARCHAR2(500), A_TEAMNAMES VARCHAR2(500), A_GOALS NUMBER, A_BALL_POSSESSION NUMBER, A_GOAL_ATTEMPTS NUMBER, A_SHOTS_ON_GOAL NUMBER, A_SHOTS_OFF_GOAL NUMBER, A_BLOCKED_SHOTS NUMBER, A_FREE_KICKS NUMBER, A_CORNER_KICKS NUMBER, A_OFFSIDES NUMBER, A_GOALKEEPER_SAVES NUMBER, A_FOULS NUMBER, A_YELLOW_CARDS NUMBER, A_RED_CARDS NUM

In [ ]:
connection = connect_to_oracle()
df = show_table(connection, "LL_2324")
# check for duplicate rows in the pd df
df

DatabaseError: ORA-00942: table or view "ADMIN"."EPL_2023" does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/

In [ ]:
# # group df by H_Ganmeinfo and H_Teamnames
# df_grouped = df.groupby(['H_GAMEINFO']).size().reset_index(name='count')
# df_grouped

False

In [ ]:
# delete_table(connection, "EXAMPLE_TABLE")

Error deleting table EXAMPLE_TABLE: ORA-00942: table or view "ADMIN"."EXAMPLE_TABLE" does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/


In [ ]:
[x] htdatan check tables from current season on false data!

[ ] add xg tables to oracle db

[ ] change updation of new season to update directly into oracle cloud db!!

[ ] check gpt for alarm if data stored gets to big there!
    
